# Import Section
---

In [1]:
"""
This script provides functionality for cropping audio files. It includes the following imports:
The script is intended to be run in a Jupyter notebook environment and uses interactive widgets to facilitate user interaction.
"""

import os
import math
from pathlib import Path
import shutil
import glob

import matplotlib.pyplot as plt
import numpy as np
from scipy.io.wavfile import read, write

from ipywidgets import interact, interact_manual
from IPython.display import clear_output
from pydub import AudioSegment
from ipyfilechooser import FileChooser
import ipywidgets as widgets
from ipywidgets import AppLayout, Button, Layout, HBox, GridBox
import auditok

from pydub import AudioSegment
import pydub

c:\ProgramData\miniforge3\envs\NuEdgeWise_env\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


# Audio Process Section
---
- Show the audio wav file fugure
- Processing the spliting wav file automatically

In [2]:
class AudioFilep:  # father class
    """
    A class to represent an audio file and perform operations related to it.
    Attributes:
    -----------
    folder : str
        The folder where the audio file is located.
    filename : str
        The name of the audio file.
    filepath : str
        The full path to the audio file.
    """

    def __init__(self, folder, filename):
        self.folder = folder
        self.filename = filename
        self.filepath = os.path.join(folder, filename)

    def create_tag_folder(self, tag_name):
        """
        Creates a folder with the specified tag name in the current working directory.
        If the folder already exists or an error occurs during creation, an error message is printed and folder creation is skipped.
        """
        dir_path = os.path.join(os.getcwd(), tag_name)
        try:
            os.mkdir(dir_path)
        except OSError as error:
            print(error)
            print("skip create")

        print(os.getcwd())


class SplitWavAudionAutoV2(AudioFilep):
    """
    A class to automatically slice audio files into smaller segments based on amplitude.
    Attributes:
    -----------
        show_plots (bool): Flag to indicate whether to show plots of the slices.
        rate (int): Sample rate of the audio file.
        data (numpy.ndarray): Audio data.
        tag (str): Path to the tag folder for saving slices.
    """

    def __init__(self, folder, filename, tagfolder, show_plots):
        self.show_plots = show_plots
        AudioFilep.__init__(self, folder, filename)
        print("Auto slice V2")
        self.rate, self.data = read(self.filepath)
        self.create_tag_folder("dataset")
        self.tag = os.path.join("dataset", tagfolder)
        self.create_tag_folder(self.tag)
        print(f"Sample rate: {self.rate} Hz")
        print(f"Data type: {self.data.dtype}")

    def btach_process(self):
        """
        Processes all files in the specified folder by performing the following steps:
        """
        file_lists = os.listdir(self.folder)
        for _, v in enumerate(file_lists):
            self.filename = v
            self.filepath = os.path.join(self.folder, self.filename)
            print(self.filepath)
            self.rate, self.data = read(self.filepath)
            self.auto_slice(self.get_auto_slice_array())

    def get_auto_slice_array(self):
        """
        Calculate and return the start and end indices of slices in the audio data based on a sliding window approach.
        The function uses a sliding window to calculate the sum of absolute values of the audio data within the window.
        It then identifies slices where the sum exceeds a threshold (mean + standard deviation) and determines the
        start and end indices of these slices.
        Returns:
            list of tuples: A list of tuples where each tuple contains the start and end indices of a slice.
        """
        factor = 1  # control window size
        win_size = self.rate  # training data's format, 1(s)
        win_u = (int)((self.rate / 10) * factor)  # windows is 0.1(s) 16000/1600
        s_r_factor = 0.5  # search range factor, too long may find the next slice part, too short may not find the real max val.

        # create window's array for latter calculate #
        sum_array = np.zeros((len(self.data) - win_u + 1), dtype=int)  # use 0.1(s) windows size to capture/calculate
        data_abs = np.absolute(self.data)
        for idx, _ in np.ndenumerate(sum_array):
            sum_array[idx[0]] = np.sum(data_abs[idx[0] : (idx[0] + win_u)])

        return self.__process_slice(sum_array, win_size, s_r_factor)

    def __process_slice(self, sum_array, win_size, s_r_factor):

        # define parameters #
        mean_d_win = sum_array.mean()
        std_win = sum_array.std()
        temp_max = 0
        temp_max_idx = 0
        start = 0
        end = 0
        slice_idx = 0
        slice_info = []

        # find the slice's start&end idx and save all in slice_info array #
        for idx, val in np.ndenumerate(sum_array):
            if (val > mean_d_win + 1 * std_win) & (idx[0] > end):  # first time trig  # only check over the previous lat cut point

                for i in range(idx[0], idx[0] + int(win_size * s_r_factor)):  # find the max window value in the range
                    if i < len(sum_array) - 1:
                        if temp_max < sum_array[i]:
                            temp_max = sum_array[i]
                            temp_max_idx = i

                if (temp_max_idx - win_size / 2 + 1) < 0:  # decide the start & end slice indexs
                    start = 0
                    end = win_size - 1
                elif (temp_max_idx + win_size / 2) > (len(self.data) - 1):
                    start = len(self.data) - win_size
                    end = len(self.data) - 1
                else:
                    start = (int)(temp_max_idx - win_size / 2 + 1)
                    end = (int)(temp_max_idx + win_size / 2)

                temp_max = 0
                slice_idx = slice_idx + 1
                slice_info.append((start, end))

        return slice_info

    def auto_slice(self, slice_a):
        """
        Automatically slices the audio data and plots the slices.
        Parameters:
        slice_a (list of tuples): A list of tuples where each tuple contains the start and end indices of a slice.
        This function performs the following steps:
        1. Plots the entire audio data with slice boxes.
        2. Saves each slice as a separate WAV file.
        3. Optionally, plots each slice if `self.show_plots` is True.
        The saved WAV files are named based on the original filename with an appended index.
        """

        time = np.arange(0, len(self.data)) / self.rate
        plt.figure(figsize=(20, 10))

        plot_a = plt.subplot(211)
        plot_a.plot(time, self.data)
        plot_a.set_ylabel("Amplitude")
        plot_a.set_xlim(0, len(self.data) / self.rate)

        # draw the slice boxes #
        max_data = self.data.max()
        for i, v in enumerate(slice_a):
            if i % 2 == 1:
                plot_a.vlines(v[0] / self.rate, -max_data - 5000, max_data + 5000, color="green")
                plot_a.vlines(v[1] / self.rate, -max_data - 5000, max_data + 5000, color="green")
                plot_a.hlines(-max_data - 5000, v[0] / self.rate, v[1] / self.rate, color="green")
                plot_a.hlines(max_data + 5000, v[0] / self.rate, v[1] / self.rate, color="green")
            else:
                plot_a.vlines(v[0] / self.rate, -max_data - 5000, max_data + 5000, color="red")
                plot_a.vlines(v[1] / self.rate, -max_data - 5000, max_data + 5000, color="red")
                plot_a.hlines(-max_data - 5000, v[0] / self.rate, v[1] / self.rate, color="red")
                plot_a.hlines(max_data + 5000, v[0] / self.rate, v[1] / self.rate, color="red")

        self.__save_sliced_wav(slice_a)

    def __save_sliced_wav(self, slice_a):
        if self.show_plots:
            plts_len = len(slice_a)
            row = math.ceil(plts_len / 2)
            col = 2

            _, axs = plt.subplots(row, col, figsize=(15, 4 * row))

        for i, v in enumerate(slice_a):  # draw the all single slice wav
            if self.show_plots:
                time = np.arange(v[0], v[1]) / self.rate
                if row > 1:
                    axs[i // 2, i % 2].plot(time, self.data[v[0] : v[1]])
                    axs[i // 2, i % 2].set_ylabel("Amplitude")
                else:
                    axs[i].plot(time, self.data[v[0] : v[1]])  # when 1 row, the axs is 1-D
                    axs[i].set_ylabel("Amplitude")

            # save the slice data
            name = self.filename.split("_nohash")[0]
            split_fn = name + "_" + str(i) + "_nohash"
            file_loc = os.path.join(os.getcwd(), self.tag, split_fn)
            if v[1] == len(self.data) - 1:
                end_s = v[1]
                start_s = v[0] - 1
            else:
                end_s = v[1] + 1
                start_s = v[0]

            write(f"{file_loc}.wav", self.rate, self.data[start_s:end_s])  # transfer from int32 to int16
            print(f"Region {i}: {(v[0]/self.rate):.3f}s -- {(v[1]/self.rate):.3f}s. Save as:{file_loc}")


class SplitWavAudionAuto(AudioFilep):
    """
    A class to automatically split and process WAV audio files.
    Attributes:
    -----------
    audio_reg : auditok.AudioRegion
        The loaded audio region from the file.
    tag : str
        The folder where the tagged audio regions will be saved.
    """

    def __init__(self, folder, filename, tagfolder):
        AudioFilep.__init__(self, folder, filename)
        print("Can only process int16 file")
        self.audio_reg = auditok.load(self.filepath)
        self.tag = tagfolder
        self.create_tag_folder(tagfolder)

    def draw(self):
        """
        Draws a plot of the audio region.
        """
        self.audio_reg.plot()

    def auto_slice(self):
        """
        Automatically slices the audio into regions based on specified parameters and saves each region as a separate file.
        The function uses the `split_and_plot` method of the `audio_reg` object to divide the audio into regions.
        Each region is then saved as a .wav file in the current working directory under a folder named after `self.tag`.
        The function prints the start and end times of each region and the filename where the region is saved.
        """
        audio_target = self.audio_reg.split_and_plot(
            min_dur=0.1,
            max_dur=3,
            max_silence=0.5,
            energy_threshold=45,
            analysis_window=0.01,
        )

        for i, r in enumerate(audio_target):
            print("Region {i}: {r.meta.start:.3f}s -- {r.meta.end:.3f}s".format(i=i, r=r))

            filename = r.save(os.path.join(os.getcwd(), self.tag, "region_{meta.start:.3f}-{meta.end:.3f}.wav"))
            print(f"Save as：{filename}")


class SplitWavAudioMubin(AudioFilep):
    """
    A class to handle splitting of WAV audio files.
    Attributes:
    -----------
    folder : str
        The folder where the audio file is located.
    filename : str
        The name of the audio file.
    tagfolder : str
        The folder where the split audio files will be saved.
    audio : AudioSegment
        The loaded audio file.
    tag : str
        The tag folder name
    """

    def __init__(self, folder, filename, tagfolder):
        AudioFilep.__init__(self, folder, filename)
        print(self.filepath)
        self.audio = AudioSegment.from_wav(self.filepath)
        self.tag = tagfolder
        self.create_tag_folder(tagfolder)

    def get_duration(self):
        """
        Get the duration of the audio.
        This method prints and returns the duration of the audio in seconds.
        Returns:
            float: The duration of the audio in seconds.
        """
        print(self.audio.duration_seconds)
        return self.audio.duration_seconds

    def single_split(self, from_sec, to_sec, split_filename):
        """
        Splits the audio file from a specified start time to end time and exports the split segment as a new file.
        Args:
            from_sec (int): The start time in seconds from which to begin the split.
            to_sec (int): The end time in seconds at which to end the split.
            split_filename (str): The name of the file to save the split audio segment.
        """
        t1 = from_sec * 1000
        t2 = to_sec * 1000
        split_audio = self.audio[t1:t2]
        file_loc = os.path.join(os.getcwd(), self.tag, split_filename)
        split_audio.export(file_loc, format="wav")

    def multiple_split(self, sec_per_split, leap):
        """
        Splits the audio file into multiple segments.

        Args:
            sec_per_split (int): The duration (in seconds) of each split segment.
            leap (int): The number of seconds to skip between each split segment.
        """
        total_secs = math.ceil(self.get_duration())
        for i in range(0, total_secs, sec_per_split + leap):
            split_fn = str(i) + "_" + self.filename
            self.single_split(i, i + sec_per_split, split_fn)
            print(str(i) + " Done")


class DrawWav:
    """
    A class to handle and visualize WAV audio files.
    Attributes:
    -----------
    folder : str
        The folder path where the WAV file is located.
    filename : str
        The name of the WAV file.
    filepath : str
        The full path to the WAV file.
    rate : int
        The sample rate of the WAV file.
    data : numpy.ndarray
        The audio data read from the WAV file.
    ch : int
        The number of channels in the audio data (1 for mono, 2 for stereo).
    """

    def __init__(self, folder, filename):
        self.folder = folder
        self.filename = filename
        self.filepath = os.path.join(folder, filename)
        self.rate, self.data = read(self.filepath)
        self.ch = 1 if self.data.size / self.data.shape[0] == 1 else 2

    def print_wavdata(self):
        """
        Prints the details of the WAV data including sample rate, data type, duration in seconds,
        number of channels, and data length.
        """
        print(f"Sample rate: {self.rate} Hz")
        print(f"Data type: {self.data.dtype}")
        print(f"Data Seconds: {self.data.shape[0] / self.rate} s")
        print(f"Number of channels: {self.ch}")
        print(f"Data length: {len(self.data)}")

    def stereo2mono(self):
        """
        Converts stereo audio data to mono by averaging the two channels.
        Returns:
            numpy.ndarray: A 1D NumPy array containing the mono audio data.
        """
        self.data = self.data.astype(float)
        w_data = self.data.sum(axis=1) / 2
        return w_data

    def plot_wave(self):
        """
        Plots the waveform and spectrogram of the audio data.
        This method plots the time domain representation and the frequency domain
        representation (spectrogram) of the audio data. If the audio data is stereo,
        it will be converted to mono before plotting.
        """

        if self.ch == 2:
            w_data = self.stereo2mono()
        else:
            w_data = self.data

        # Time data
        time = np.arange(0, len(w_data)) / self.rate
        plt.figure(figsize=(15, 5))

        # draw time domain
        plot_a = plt.subplot(211)
        plot_a.plot(time, w_data)
        plot_a.set_ylabel("Amplitude")
        plot_a.set_xlim(0, len(w_data) / self.rate)

        # draw frequency domain
        plot_b = plt.subplot(212)
        plot_b.specgram(w_data, NFFT=1024, Fs=self.rate, noverlap=900)
        plot_b.set_ylabel("Frequency")
        plot_b.set_xlabel("Time")

        plt.show()

class SplitSingleWavAuto(AudioFilep):
    """
    """

    def __init__(self, folder_path, fdir, file, window, min_silence_len, silence_thresh, pad_to_user_window): 
        self.folder_path = folder_path
        self.folder = fdir
        self.filename = file
        self.autoslice_output_path = Path(os.getcwd()) / "dataset" / Path(self.folder)
        self.window = window
        self.min_silence_len = min_silence_len
        self.silence_thresh = silence_thresh
        self.pad_to_user_window = pad_to_user_window 

    def btach_process(self):
        for wav_file in glob.glob(os.path.join(self.folder_path, "*.wav")):
            print(wav_file)
            win_idxs = self.get_single_slice_range(wav_file, self.min_silence_len, self.silence_thresh, self.window, self.autoslice_output_path)

    def draw_single_slice_range(self, original_data, fr, slice_idx_list, file_name):
        """
        Plots the waveform of the original data and overlays slice ranges as colored boxes.
        Parameters:
        -----------
        original_data : numpy.ndarray
            The audio data to be plotted.
        fr : int or float
            The sampling frequency of the audio data.
        slice_idx_list : list of tuples
            A list of slice index ranges, where each tuple contains two integers 
            representing the start and end indices of a slice. Alternating slices 
            are colored green and red.
        file_name : str
            The title of the plot, typically the name of the file being visualized.
        """
        time = np.arange(0, len(original_data)) / fr  # Create a time array
        plt.figure(figsize=(20, 10))
        plot_a = plt.subplot(211)
        plot_a.plot(time, original_data)
        plot_a.set_ylabel("Amplitude")
        plot_a.set_xlim(0, len(original_data) / fr)
        plot_a.set_title(str(file_name))
        # draw the slice boxes #
        max_data = original_data.max()
    
        for i, v in enumerate(slice_idx_list):
            if i % 2 == 1:
                plot_a.vlines(v[0] / 1000, -max_data - 5000, max_data + 5000, color="green")
                plot_a.vlines(v[1] / 1000, -max_data - 5000, max_data + 5000, color="green")
                plot_a.hlines(-max_data - 5000, v[0] / 1000, v[1] / 1000, color="green")
                plot_a.hlines(max_data + 5000, v[0] / 1000, v[1] / 1000, color="green")
            else:
                plot_a.vlines(v[0] / 1000, -max_data - 5000, max_data + 5000, color="red")
                plot_a.vlines(v[1] / 1000, -max_data - 5000, max_data + 5000, color="red")
                plot_a.hlines(-max_data - 5000, v[0] / 1000, v[1] / 1000, color="red")
                plot_a.hlines(max_data + 5000, v[0] / 1000, v[1] / 1000, color="red")
    
    
    def save_single_slice_range(self, ori_audio, slice_idx_list, sliced_audio_path):
        """
        Save a specific range of an audio file as a new audio file.
        This function takes an original audio object, extracts a specific slice
        of the audio based on the provided index range, and saves the sliced
        audio to the specified file path in WAV format.
        Args:
            ori_audio: An audio object representing the original audio data.
            slice_idx_list (list of tuples): A list containing tuples where each
                tuple specifies the start and end indices of the slice to extract.
                Only the first tuple in the list is used.
            sliced_audio_path (str): The file path where the sliced audio will be
                saved in WAV format.
        """
        ori_audio = ori_audio[slice_idx_list[0][0]:slice_idx_list[0][1]]
    
        ori_audio.export(sliced_audio_path, format="wav")
    
    
    def get_single_slice_range(self, audio_path, min_silence_len, silence_thresh, user_set_window, output_save_path, pad_to_user_window=True):
        """
        Get the single slice range of the audio file.
        """
        # Create the output directory if it doesn't exist
        output_save_path.mkdir(exist_ok=True)
    
        # Load the audio file
        audio_segment = AudioSegment.from_file(audio_path, format="wav")
        channel_count = audio_segment.channels
        if channel_count > 1:
            audio_segment = audio_segment.set_channels(1)  # Convert to mono
        frames_per_second = audio_segment.frame_rate
        if frames_per_second != 16000:
            audio_segment = audio_segment.set_frame_rate(16000)
        sample_width = audio_segment.sample_width
        if sample_width != 2:
            audio_segment = audio_segment.set_sample_width(2)  # Convert to 16-bit
        print(f"channel_num: {channel_count}, frame_rate: {frames_per_second}, length: {(len(audio_segment) / 1000.0)}, sample_width: {sample_width}")
    
        # assert len(audio_segment) >= user_set_window, f"user set window should be smaller than the size of wav file."
        if len(audio_segment) > user_set_window:
    
            not_silence_ranges = pydub.silence.detect_nonsilent(audio_segment, min_silence_len=min_silence_len, silence_thresh=silence_thresh, seek_step=1)
            print(f"original slice range (ms): {not_silence_ranges}")
    
            # Combine the ranges list's first value with the last value
            single_slice_range = []
            if len(not_silence_ranges) > 1:
                single_slice_range = (not_silence_ranges[0][0], not_silence_ranges[-1][1])
            elif len(not_silence_ranges) == 1:
                single_slice_range = not_silence_ranges[0]
            else:
                print("No slice range found")
    
            # update the not_silence_ranges to be more accurate & basing on user setting
            slice_window = single_slice_range[1] - single_slice_range[0]
            if slice_window > user_set_window:
                # if the slice window is too long, we need to split it into two, first from begin, second from end
                not_silence_ranges = [(single_slice_range[0], single_slice_range[0] + user_set_window), (single_slice_range[1] - user_set_window, single_slice_range[1])]
                not_silence_ranges = [not_silence_ranges[0]]
            else:
                # if the slice window is too short, we need to pad it before&after
                pad_size = np.ceil((user_set_window - slice_window) / 2)
                pad_idx_begin = 0
                pad_idx_end = 0
    
                # pad before
                if single_slice_range[0] - pad_size > 0:
                    pad_idx_begin = single_slice_range[0] - pad_size
                else:
                    pad_idx_begin = 0
                    # update the pad size
                    pad_size = (user_set_window - slice_window) - single_slice_range[0]
    
                # pad after
                if single_slice_range[1] + pad_size < len(audio_segment):
                    pad_idx_end = single_slice_range[1] + pad_size
                else:
                    pad_idx_end = len(audio_segment)
                    # update the begin index
                    pad_size = (user_set_window - slice_window) - (len(audio_segment) - single_slice_range[1])
                    pad_idx_begin = single_slice_range[0] - pad_size
                # update the not_silence_ranges
                not_silence_ranges = [(pad_idx_begin, pad_idx_end)]
    
            print(f"updated slice range (ms): {not_silence_ranges}")
            self.draw_single_slice_range(np.array(audio_segment.get_array_of_samples()), frames_per_second, not_silence_ranges, Path(audio_path).name)
            sliced_audio_path = os.path.join(output_save_path, (Path(audio_path).stem + "_" + str(user_set_window / 1000) + "s" + "_" + "0" + "_nohash" + Path(audio_path).suffix))
            self.save_single_slice_range(audio_segment, not_silence_ranges, sliced_audio_path)
        else:
            if pad_to_user_window:
                padding_needed = np.ceil((user_set_window - len(audio_segment)) / 2)
                silence = AudioSegment.silent(duration=padding_needed)
                # Add silence to the end of the audio
                padded_audio = silence + audio_segment + silence
                not_silence_ranges = [(0, len(padded_audio))]
    
                print(f"audio length is shorter than user set window, padding!! updated slice range (ms): {not_silence_ranges}")
                self.draw_single_slice_range(np.array(padded_audio.get_array_of_samples()), frames_per_second, not_silence_ranges, Path(audio_path).name)
                sliced_audio_path = os.path.join(output_save_path, (Path(audio_path).stem + "_" + str(user_set_window / 1000) + "s" + "_" + "0" + "_nohash" + Path(audio_path).suffix))
                self.save_single_slice_range(padded_audio, not_silence_ranges, sliced_audio_path)
            else:
                print(f"audio length is shorter than user set window, {audio_path} SKIP!")
                not_silence_ranges = None
    
        return not_silence_ranges               

# Widgets Control Section
---

In [3]:
output = widgets.Output()  # interact output


class InitAudio:
    """
    A class to initialize and interact with audio files in a directory.

    Attributes:
    -----------
    mypath : str
        The current working directory path.
    dirpath : str
        The directory path of the first level.
    dirnames : list
        The list of directory names in the first level.
    filenames : list
        The list of filenames in the first level.

    Methods:
    --------
    get_firstlevel():
        Retrieves the first level of directories and files.

    create_expanded_button(description, button_style):
        Creates an expanded button with the given description and style.

    copy_allfiles(src_folder, dst_folder):
        Copies all files from the source folder to the destination folder.

    interact_block_audio():
        Creates an interactive block for audio file selection and processing.
    """

    def __init__(self):
        self.mypath = os.getcwd()
        self.dirpath, self.dirnames, self.filenames = self.get_firstlevel()
        self.file = None
        self.folder_loc = None
        self.fdir = None

    def get_firstlevel(self):  # get a list of folders
        """
        Retrieves the first level of directories and files in the specified path.
        Returns:
            tuple: A tuple containing:
                - dirpath (str): The path to the directory.
                - dirnames (list): A list of directory names in the first level.
                - filenames (list): A list of file names in the first level.
        """
        dirpath, dirnames, filenames = next(os.walk(self.mypath))

        return dirpath, dirnames, filenames

    def create_expanded_button(self, description, button_style):
        """
        Create a Button widget with the specified description and style.
        Parameters:
        description (str): The text to display on the button.
        button_style (str): The style of the button (e.g., 'primary', 'success', 'info', 'warning', 'danger').
        Returns:
        Button: A Button widget with the specified description and style.
        """
        return Button(
            disabled=False,
            description=description,
            button_style=button_style,
            layout=Layout(height="auto", width="auto"),
        )

    def copy_allfiles(self, src_folder, dst_folder):
        """
        Copies all files from the source folder to the destination folder.
        Args:
            src_folder (str): The path to the source folder.
            dst_folder (str): The path to the destination folder.
        """
        copy_num = 0
        for file_name in os.listdir(src_folder):
            source = os.path.join(src_folder, file_name)
            destination = os.path.join(dst_folder, file_name)
            # copy only files
            if os.path.isfile(source):
                shutil.copy(source, destination)
                copy_num = copy_num + 1
        print(f"Copy finish, total {copy_num} files")

    def on_button_clicked_event_a(self, b):
        """
        Event handler for button click event 'a'.
        """
        with output:
            clear_output()
            draw_wav = DrawWav(self.folder_loc, self.file)
            draw_wav.print_wavdata()
            draw_wav.plot_wave()

    def on_button_clicked_event_b(self, b):
        """
        Event handler for button click event.
        This function is triggered when a button is clicked. It clears the current output,
        prints a message to prompt the user to set the slice parameters, and then creates
        an interactive widget for splitting a WAV file.
        """
        with output:
            clear_output()
            print("Please set the slice!!")

            def act_split_wav(sec_per_split, leap, tag_name):
                split_wav = SplitWavAudioMubin(self.folder_loc, self.file, tag_name)
                split_wav.multiple_split(sec_per_split, leap)
                return (sec_per_split, leap, tag_name)

            evt = interact_manual(
                act_split_wav,
                sec_per_split=widgets.IntSlider(min=0, max=10, step=1, value=10, description="Seconds/Slice"),
                leap=widgets.IntSlider(min=0, max=10, step=1, value=1, description="Gap"),
                tag_name=widgets.Text(value="test_slice", description="Label"),
            )
            evt.widget.children[3].description = "Start Slice"  # because there are 3 parameter of the evt
            evt.widget.children[3].button_style = "success"

    def on_button_clicked_event_c(self, b):
        """
        Event handler for button click event.
        This function is triggered when a button is clicked. It clears the current output and
        sets up an interactive manual widget to perform automatic audio slicing.
        """
        with output:
            clear_output()

            def act_auto_split(tag_name, batch_en, show_plots):
                if not batch_en:
                    split_auto_v2 = SplitWavAudionAutoV2(self.folder_loc, self.file, tag_name, show_plots)
                    split_auto_v2.auto_slice(split_auto_v2.get_auto_slice_array())
                else:
                    print("auto slice batch")
                    split_auto_v2 = SplitWavAudionAutoV2(self.folder_loc, self.file, tag_name, show_plots)
                    split_auto_v2.btach_process()

            evt = interact_manual(
                act_auto_split,
                tag_name=widgets.Text(value="test_slice", description="Label"),
                batch_en=widgets.Checkbox(
                    value=True,
                    disabled=False,
                    indent=False,
                    description="Batch Enable",
                ),
                show_plots=widgets.Checkbox(
                    value=False,
                    disabled=False,
                    indent=False,
                    description="Show Plots",
                ),
            )
            evt.widget.children[3].description = "Start Slice"  # because there are 3 parameter of the evt
            evt.widget.children[3].button_style = "success"

    def on_button_clicked_event_d(self, b):
        """
        Handles the event when a button is clicked.
        """
        with output:
            clear_output()
            path_fc = os.path.abspath("dataset")  # The processed data are in /dataset
            fc = FileChooser(path_fc)
            fc.show_only_dirs = True
            fc.title = "<b><font color='lightgreen'><font size=4>Choose train label folder.</b>"
            display(fc)
            path_f_copy = os.path.abspath("..")  # The train program is in ../ML_kws_tflu
            path_f_copy = os.path.join(path_f_copy, "ML_kws_tflu", "tmp", "speech_dataset")

            def act_copy_folder():
                print(f"Selected folder: {fc.selected_path}")
                copied_folder_name = Path(fc.selected_path).parts[-1]
                copied_dst = os.path.join(path_f_copy, copied_folder_name)
                if not os.path.isdir(copied_dst):  # check if the folder exists or not, if not creat it.
                    os.makedirs(copied_dst)
                print(f"Copy to: {path_f_copy}")
                self.copy_allfiles(fc.selected_path, copied_dst)

            evt = interact_manual(act_copy_folder)
            evt.widget.children[0].description = "Start Copy"  # because there are 3 parameter of the evt
            evt.widget.children[0].button_style = "success"

    def on_button_clicked_event_e(self, b):
        """
        """
        with output:
            clear_output()

            def act_auto_single_split(window, min_silence_len, silence_thresh, pad_to_user_window):
                
                print("auto single slice batch")
                split_single_auto = SplitSingleWavAuto(self.folder_loc, self.fdir, self.file, window, min_silence_len, silence_thresh, pad_to_user_window)
                split_single_auto.btach_process()

            evt = interact_manual(
                act_auto_single_split,
                window = widgets.BoundedIntText(min=500, max=6000, step=500, value=1000, description="USER SET WINDOW", 
                                                style={'description_width': '200px'}, layout=widgets.Layout(width='300px')),
                min_silence_len = widgets.BoundedIntText(min=100, max=6000, step=100, value=300, description="MIN SILENCE LEN", 
                                                         style={'description_width': '200px'}, layout=widgets.Layout(width='300px')),
                silence_thresh = widgets.IntText(value=-35, description="SILENCE THRESH", 
                                                 style={'description_width': '200px'}, layout=widgets.Layout(width='300px')),
                pad_to_user_window = widgets.Checkbox(value=True,description='PAD TO USER WINDOW', 
                                                    ),
                layout=Layout(width='100%', height='80px')
                )
            evt.widget.children[4].description = "Start Slice"  # because there are 3 parameter of the evt
            evt.widget.children[4].button_style = "success"        

    def interact_block_audio(self):
        """
        Creates an interactive widget-based interface for selecting and processing audio files.
        This method provides a graphical interface using Jupyter widgets to:
        - Select a folder and file from the directory.
        - Display buttons for various audio processing tasks:
            - Run Wav Plot: Plots the waveform of the selected audio file.
            - Slice Wav: Manually slices the audio file based on user-defined parameters.
            - Slice Wav Auto: Automatically slices the audio file with optional batch processing and plotting.
            - Copy to where (for training use): Copies the processed audio files to a specified training directory.
        The interface dynamically updates the available files based on the selected folder and provides
        interactive controls for each processing task.
        """

        dirnames = self.dirnames
        choose_folder = widgets.Dropdown(options=dirnames)
        choose_file = widgets.Dropdown(options=os.listdir(choose_folder.value))

        def update_choose_file(*args):  # Updates the choose_file options based on choose_folder value
            choose_file.options = os.listdir(choose_folder.value)

        choose_folder.observe(update_choose_file, "value")  # Tie the choose_file options to choose_folder value

        def show_workarea(fdir, file):
            self.file = file
            self.folder_loc = os.path.join(os.getcwd(), fdir)
            self.fdir = fdir
            print(f"Choosed folder: {self.folder_loc}")
            print(f"Choosed file: {self.file}")

            button_a = self.create_expanded_button("Run Wav Plot", "success")
            button_b = self.create_expanded_button("Slice Wav", "info")
            button_c = self.create_expanded_button("Slice Wav Auto", "warning")
            button_d = self.create_expanded_button("Copy to where(for training use)", "info")
            button_e = self.create_expanded_button("Slice Single Wav Auto", "warning")

            display(  # show the buttom and output
                AppLayout(
                    header=button_e,
                    left_sidebar=button_a,
                    center=button_b,
                    right_sidebar=button_c,
                    footer=button_d,
                    pane_widths=[3, 3, 3],
                    pane_heights=["40px", "40px", "40px"],
                ),
                output,
            )

            button_a.on_click(self.on_button_clicked_event_a)

            button_b.on_click(self.on_button_clicked_event_b)

            button_c.on_click(self.on_button_clicked_event_c)

            button_d.on_click(self.on_button_clicked_event_d)

            button_e.on_click(self.on_button_clicked_event_e)

        if choose_folder.value is not None:
            _ = interact(show_workarea, fdir=choose_folder, file=choose_file)
        else:
            print(f"Please at least a folder in {os.getcwd()}!!!")

# Run Section
- The detail description is here [meaning](#id-control-intro)

In [4]:
audio_proj = InitAudio()
audio_proj.interact_block_audio()

interactive(children=(Dropdown(description='fdir', options=('.ipynb_checkpoints', 'dataset', 'raw1', '功能词-唤醒30…

<a id="id-control-intro"></a>
# Control Introduction
---

- `fdir`: the folders in `ML_audio_aq` folder, you can choose any one of them.
- `file`: the files in the choosen folder, you can choose any one of them.
- `Run Wav Plot`: show the *.wav's figure.
- `Slice Wav`: manually slice the single wav file(old).
- `Slice Wav Auto`: smartly slice the single or whole folder's wav files.

- In `Slice Wav Auto`:
    1. Label is the folder's name which save all the sliced wav files. You should follow the training's label(answer).
    2. `Btach Enable`: slice the whole folder's wav files.
    3. `Show Plots`: show each sliced small plots.
    4. `Start Slice`: you can click this buttom after all setting is done.
    5. (recommend)(will update) The output `Label` folder is saved in `dataset`. You can copy/move the folders to training step's train_data_folder, for example: in `ML_kws_tflu\tmp\speech_dataset`

---
- In `Slice Single Wav Auto`:
    - `FOLDER_PATH`: 欲切割之目標資料夾. 切割後的資料夾在 `dataset\{user set folder_path}`
    - `USER_SET_WINDOW`: 欲設定之固定窗口大小.
    - `MIN_SILENCE_LEN`: 持续多少时间可认定为静默
    - `SILENCE_THRESH`: 声音大小小于多少时可认定为静默，默认值为-16dBFS
    - `PAD_TO_USER_WINDOW`: True or False. 如果欲切割wav檔長度小於設定之固定窗口大小，決定是否在原檔前後padding
    - 建議: 如果自動分割效果不太好，再自行調整`min_silence_len`和`silence_thresh`    